<a href="https://colab.research.google.com/github/kathdevx/emotionalAI/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install soundfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir = '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/'
partitions = '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/meta-info/Partitions.txt'

import soundfile as sf
import glob
train = {'fname' : [], 'data': []}
test = {'fname' : [], 'data': []}
val = {'fname' : [], 'data': []}

part_file = open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/meta-info/Partitions.txt')
c = 20
for line in part_file:
    idx = line.find(';')
    if line[:idx] == 'Train':
        # train['fname'].append(line[:-1])
        train['fname'].append(line[idx+2:-1])
        c -= 1
    if not c:
        break

c = 20
for filename in glob.glob(dir+'*.wav'):
    idx = filename.rfind('/')
    fname = filename[idx+1:]
    if fname in train['fname']:
        f = sf.SoundFile(filename)
        train['data'].append(filename)
        c -= 1
    if not c:
        break

print(train)

{'fname': ['MSP-PODCAST_0023_0048.wav', 'MSP-PODCAST_0023_0205.wav', 'MSP-PODCAST_0023_0206.wav', 'MSP-PODCAST_0023_0229.wav', 'MSP-PODCAST_0023_0246.wav', 'MSP-PODCAST_0023_0247.wav', 'MSP-PODCAST_0023_0281.wav', 'MSP-PODCAST_0023_0406.wav', 'MSP-PODCAST_0023_0589.wav', 'MSP-PODCAST_0023_0018.wav', 'MSP-PODCAST_0023_0006.wav', 'MSP-PODCAST_0023_0044.wav', 'MSP-PODCAST_0023_0053.wav', 'MSP-PODCAST_0023_0078.wav', 'MSP-PODCAST_0023_0144.wav', 'MSP-PODCAST_0023_0217.wav', 'MSP-PODCAST_0023_0218.wav', 'MSP-PODCAST_0023_0248.wav', 'MSP-PODCAST_0023_0285.wav', 'MSP-PODCAST_0023_0327.wav'], 'data': ['/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0006.wav', '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0018.wav', '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0044.wav', '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data

In [ ]:
train

{'data': ['/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0006.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0018.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0044.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0078.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0053.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0048.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0144.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_0023_0218.wav',
  '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/MSP-PODCAST_

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import librosa
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import nltk

def load_model():
    tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    return tokenizer, model

def correct_sentence(input_text):
    sentences = nltk.sent_tokenize(input_text)
    return (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

def asr_transcript(input_file):

    tokenizer, model = load_model()
    
    speech, fs = sf.read(input_file)

    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]

    if fs != 16000:
        speech = librosa.resample(speech, fs, 16000)

    input_values = tokenizer(speech, return_tensors="pt").input_values
    logits = model(input_values).logits
    
    predicted_ids = torch.argmax(logits, dim=-1)
    
    transcription = tokenizer.decode(predicted_ids[0])

    return correct_sentence(transcription.lower())


In [ ]:
f = open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/sentences.txt', 'w')

sentences = []

for sample in train['data']:
    idx = sample.rfind('/')
    fname = sample[idx+1:] + ' '
    f.write(fname)
    f.write(asr_transcript(sample))
    f.write('\n')

# asr_transcript(train['data'][3])

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:419: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_

In [ ]:
f.close()